In [13]:
import numpy as np

In [369]:
def DecisonTree(object):
    def __init(self,**kwargs):
        '''
        Height: the Max Height of Tree
        '''
        self.Height = Height
        self.task = task
        self.Tree = {}
        
    def train(self,train_X,train_y):
        '''
        对于类别属性来说，有两种处理方法：
        1. 按属性中所有的值来分类来生成树（C3.0等早期分类树就是这种方式）
        2. 将类别属性One-hot 编码来这样就只有0-1变量了。（Xgboost,CART 是这种)
        3. 其实如果按照最好的离散属性分割方式，应该穷举所有的组合，找到一个最好的分割方式。H20好像有这种实践。

        对于连续属性来说，就很好办了，直接二分分类。
        属性中的值来说，一般来说，如果是（0,1）变量，那么每次分类后就可以去掉这个特征了，对于连续变量就不是了，要继续保留参与分割。

        本算法采用方法2处理离散变量。即不再区分特征是离散变量还是连续变量，这样做能降低代码复杂度，而且做到了分类和回归任务的统一处理。
        本算法仅仅生成二叉树，不生成多叉树。
        '''
        '''
        不可采用分割方式，将子数据集传入递归函数，这样会丢失feature 位置，也就没法记录分割位置了，哈哈哈
        算了，还是切割吧。简化程序处理
        '''
        self.train_X = train_X
        self.train_y = train_y
        tree = TreeGenerate(train_X,train_y)
        self.Tree = pruning(tree)

In [10]:
>>> from sklearn.datasets import load_iris
>>> from sklearn import tree
>>> iris = load_iris()
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)

In [319]:
y_train[[1,2,3,7,99]]

array([2, 2, 1, 1, 0])

In [49]:
A = np.arange(X_train.shape[1])

In [140]:
def chooseBestSplitfeature():
    return 3

In [367]:
class self():
    X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)
    max_depth = 5
    min_samples_split = 4
    min_samples_leaf = 2
    min_impurity_decrease = 0
    
    

In [576]:
def TreeGenerate(X_train,y_train):
    #X_train = self.X_train[indexes,cols]
    #y_train = self.y_train[indexes]
    best_feature,best_value = chooseBestSplitfeature(X_train,y_train)#选取最佳分裂属性，最佳分裂点
    print('best_feature,best_value::',best_feature,best_value)
    if best_feature is None:
        return best_value
    Tree = {}

    Tree['splitF'] = best_feature
    Tree['splitV'] = best_value
    left_X_train,left_y_train,right_X_train,right_y_train = binSplit(X_train,y_train,best_feature,best_value)
#     lef_cols = np.argwhere(np.logical_not((left_X_train==left_X_train[0]).all(axis=0))).flatten()
#     lef_cols = np.argwhere(np.logical_not((left_X_train==left_X_train[0]).all(axis=0))).flatten()
    Tree['left'] = TreeGenerate(left_X_train,left_y_train)
    Tree['right'] = TreeGenerate(right_X_train,right_y_train)
    return Tree
    
def chooseBestSplitfeature(X_train,y_train):
    cls,cnt = np.unique(y_train,return_counts=True)#统计y值的数量
    print('21,cls::',cls)
    if cls.shape[0]==1:#如果y值全部一样，停止分裂
        print('case 1:cls.shape[0]==1',cls)
        node = cls[0]
        return None,node
    if (X_train == X_train[0]).all():
        #如果属性值为空为空，或者X中全部样本相同，则返回y中出现最多的值
        print('case 2:== X_train[0]).all()', X_train[0])
        return None,get_NodeValue(y_train)
    if X_train.shape[0] < self.min_samples_split:# 如果X数量小于最小样本分割数，停止分裂
        print('case3 min_samples_split',X_train.shape[0])
        return None,get_NodeValue(y_train)
    
    current_error = cal_Error(y_train)
    print('current_error:',current_error)
    best_feature = 0
    best_value = get_NodeValue(y_train)
    best_error = cal_Error(y_train)
    for i in range(X_train.shape[1]):
        cls = np.unique(X_train[:,i])
        if cls.shape[0]==1:
            continue
        cls = np.delete(cls,np.argmin(cls))
        for val in cls:
            left_y_train,right_y_train = binSplit_y(X_train,y_train,i,val)
            if (left_y_train.shape[0] < self.min_samples_leaf) or\
            (right_y_train.shape[0] <self.min_samples_leaf):
                #print(left_y_train.shape,end=',')
                #print(right_y_train.shape,end=',')
                #error = cal_Error(y_train)
                pass
            else:
                error = cal_Error(left_y_train,right_y_train)
                if error < best_error:
                    #print('error:',error)
                    best_feature = i
                    best_value = val
                    best_error = error
        #print('\n')
    if current_error - best_error < self.min_impurity_decrease:
        print('case4 min_impurity_decrease',current_error - best_error)
        return None,get_NodeValue(y_train)
    
    return best_feature,best_value
            
            
def binSplit(X_train,y_train,feature,value):
    con_lt = np.where(X_train[:,feature]<value)
    con_get = np.where(X_train[:,feature]>= value)
    return X_train[con_lt],y_train[con_lt],X_train[con_get],y_train[con_get]            

def binSplit_y(X_train,y_train,feature,value):
    #print(66,feature,value,end=';')
    con_lt = np.where(X_train[:,feature]<value)
    con_get = np.where(X_train[:,feature]>= value)
    return y_train[con_lt],y_train[con_get]


def get_NodeValue(y_train):
    '''
    如果是分类任务，返回出现频率最高的一个值。
    如果是回归任务，返回数据平均值
    '''
    cls,cnt = np.unique(y_train,return_counts=True)
    return max(zip(cls,cnt),key=lambda x:x[1])[0]

def cal_Error(left_y_train,right_y_train=None):
    '''
    
    分类任务是 计算gini，熵；回归任务一般是计算MAE。
    refer:
    [1] scikit-learn. http://scikit-learn.org/stable/modules/tree.html#classification-criteria
    [2] 《机器学习》P79页4.2.3节基尼系数(周志华著作，西瓜书)
    '''
    #np.mean(y_train)
    if right_y_train is None:
        return get_gini(left_y_train)
    left_gini = get_gini(left_y_train)
    right_gini = get_gini(right_y_train)
    gini = np.array([left_gini,right_gini])
    shape_l = left_y_train.shape[0];shape_r = right_y_train.shape[0]
    weight = np.array([shape_l,shape_r])/(shape_l+shape_r)
    
    
    gini_index = (weight*gini).sum()
    
    return gini_index
    
def get_gini(y_train):
    
    lens = y_train.shape[0]
    vals,cnts = np.unique(y_train,return_counts=True)
    #print(vals,cnts,cnts/lens)
    gini = 1 - ((cnts/lens)**2).sum()# 就是这么简单...
    return gini
    
    

def pruning(Tree):
    '''
    如果你把生成的树的图化出来，你会发现有些树的非叶子节点左右子树的值一样，这样肯定是不合理的，所以要有一个剪枝的buzhou
    本函数只是为了优化树的结构，并不是用来防止过拟合的。
    '''
    if not isinstance(Tree,dict):
        return Tree
    if Tree['left'] == Tree['right']:
        return Tree['left']
    if isinstance(Tree['left'],dict):
        Tree['left'] = pruning(Tree['left'])
    if isinstance(Tree['right'],dict):
        Tree['right'] = pruning(Tree['right'])
    return Tree

In [541]:
tree = TreeGenerate(X_train,y_train)

21,cls:: [0 1 2]
current_error: 0.664489795918
best_feature,best_value:: 2 3.0
21,cls:: [0]
case 1:cls.shape[0]==1 [0]
best_feature,best_value:: None 0
21,cls:: [1 2]
current_error: 0.5
best_feature,best_value:: 2 4.8
21,cls:: [1 2]
current_error: 0.058769513315
best_feature,best_value:: 0 5.0
21,cls:: [1 2]
case3 min_samples_split 2
best_feature,best_value:: None 1
21,cls:: [1]
case 1:cls.shape[0]==1 [1]
best_feature,best_value:: None 1
21,cls:: [1 2]
current_error: 0.214158239143
best_feature,best_value:: 3 1.8
21,cls:: [1 2]
current_error: 0.5
best_feature,best_value:: 2 5.0
21,cls:: [1]
case 1:cls.shape[0]==1 [1]
best_feature,best_value:: None 1
21,cls:: [1 2]
current_error: 0.444444444444
best_feature,best_value:: 3 1.6
21,cls:: [2]
case 1:cls.shape[0]==1 [2]
best_feature,best_value:: None 2
21,cls:: [1 2]
case3 min_samples_split 3
best_feature,best_value:: None 1
21,cls:: [1 2]
current_error: 0.058769513315
best_feature,best_value:: 2 4.9
21,cls:: [1 2]
case3 min_samples_split 3


In [556]:
tree['right']['left'] == tree['right']['right']

False

In [574]:
tree2 = pruning(tree)

In [575]:
tree2

{'left': 0,
 'right': {'left': 1,
  'right': {'left': {'left': 1,
    'right': {'left': 2,
     'right': 1,
     'splitF': 3,
     'splitV': 1.6000000000000001},
    'splitF': 2,
    'splitV': 5.0},
   'right': 2,
   'splitF': 3,
   'splitV': 1.8},
  'splitF': 2,
  'splitV': 4.7999999999999998},
 'splitF': 2,
 'splitV': 3.0}

In [571]:
tree2['right']['right']['left']['right']

{'left': 2, 'right': 1, 'splitF': 3, 'splitV': 1.6000000000000001}

In [572]:
'y' == 'y'

True

In [532]:
tree['right']['right']['right']

{'left': 2, 'right': 2, 'splitF': 2, 'splitV': 4.9000000000000004}

In [533]:
tree

{'left': 0,
 'right': {'left': {'left': 1, 'right': 1, 'splitF': 0, 'splitV': 5.0},
  'right': {'left': {'left': 1,
    'right': {'left': 2,
     'right': 1,
     'splitF': 3,
     'splitV': 1.6000000000000001},
    'splitF': 2,
    'splitV': 5.0},
   'right': {'left': 2, 'right': 2, 'splitF': 2, 'splitV': 4.9000000000000004},
   'splitF': 3,
   'splitV': 1.8},
  'splitF': 2,
  'splitV': 4.7999999999999998},
 'splitF': 2,
 'splitV': 3.0}

In [492]:
a=np.array([2,5])
b=np.array([5,10])

In [509]:
a,b = binSplit_y(X_train,y_train,2,3)

In [511]:
cal_Error(a,b)

[0] [31] [ 1.]
[1 2] [37 37] [ 0.5  0.5]


0.35238095238095241

In [488]:
get_gini(y_train)

[0 1 2] [31 37 37] [ 0.2952381   0.35238095  0.35238095]


0.66448979591836732

In [473]:
A = np.array([1,1,2,2,3,3])

a,b = np.unique(A,return_counts=True)

1 - ((b/6)**2).sum()

In [521]:
chooseBestSplitfeature(X_train,y_train)

current_error: 0.664489795918
error: 0.642577030812
error: 0.63498349835
error: 0.611273080661
error: 0.603043691703
error: 0.594642857143
error: 0.591627420199
error: 0.554665414906
error: 0.488761904762
error: 0.485135355512
error: 0.473112073112
error: 0.455647573039




error: 0.455516840883
error: 0.419288728149
error: 0.380200501253
error: 0.352380952381






(2, 3.0)

In [ ]:
X_train[:,2]

In [491]:
np.unique(X_train[:,2])

array([ 1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.9,  3. ,  3.3,  3.5,
        3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,  4.5,  4.6,  4.7,
        4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,  5.6,  5.7,  5.8,
        5.9,  6. ,  6.1,  6.3,  6.6,  6.7])

In [460]:
X_train[:,0]

array([ 5.5,  6.3,  6.4,  6.6,  7.2,  5.7,  7.6,  5.6,  5.1,  7.7,  5.8,
        5.2,  5. ,  5.1,  5. ,  6.3,  4.8,  5. ,  5.1,  5.6,  5.1,  5.7,
        7.7,  4.6,  6.2,  5.7,  5.5,  6. ,  5.8,  6. ,  5.4,  6.2,  5.5,
        5.4,  5. ,  6.4,  5. ,  5. ,  5.5,  6.7,  4.9,  5.8,  5. ,  5. ,
        5.9,  5.1,  6.9,  6. ,  6.1,  7.7,  5.5,  4.4,  4.3,  6. ,  7.2,
        4.6,  5.1,  4.4,  6.3,  6.3,  4.6,  6.8,  6.3,  4.7,  6.1,  6.5,
        6.2,  7. ,  6.4,  5.1,  6.9,  5.9,  6.5,  5.7,  5.2,  6.1,  4.5,
        6.6,  5.5,  5.3,  5.6,  7.3,  6.7,  5.1,  4.9,  6.7,  7.2,  4.9,
        6.7,  4.9,  6.9,  7.4,  6.3,  5.7,  6.5,  6.3,  6.4,  5.6,  5.9,
        5.4,  6.1,  4.9,  5.8,  5.8,  7.1])

In [436]:
binSplit_y(X_train,y_train,0,4.4)

66 0 4.4


(array([0]),
 array([1, 2, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2,
        0, 1, 2, 0, 2, 2, 1, 1, 2, 1, 0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1,
        2, 1, 2, 2, 1, 0, 2, 2, 0, 0, 0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0,
        2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2,
        1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2]))

In [430]:
con_lt = np.where(X_train[:,0]<5.9)
con_get = np.where(X_train[:,0]>= 5.9)
y_train[con_lt]
#y_train[con_get]

array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 2, 0, 2, 1, 1, 0, 1,
       0, 0, 1, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 2, 0, 2, 0, 1])

In [431]:
y_train[con_get]

array([2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2,
       2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1,
       2, 1, 2])

In [432]:
np.unique(X_train[:,0])

array([ 4.3,  4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,
        5.4,  5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,
        6.5,  6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.6,
        7.7])

In [360]:
cls = np.delete(cls,np.argmin(cls))

In [362]:
for i in cls:
    print(i)

5.5


In [201]:
np.where(X_train[:,1]>3)[0].shape

(41,)

In [216]:
X_train.T.shape

(4, 105)

In [217]:
X_train.T==X_train.T[0]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, Fals

In [220]:
def myfunc(a,b):
    if (a>b): return a
    else: return b
vecfunc = np.vectorize(myfunc)
result=vecfunc([[1,2,3],[5,6,9]],[7,4,5])
print(result)


[[7 4 5]
 [7 6 9]]


In [229]:
X_train[np.unique(X_train)]

IndexError: arrays used as indices must be of integer (or boolean) type

In [238]:
A=np.array([[1,2,3,4],[1,3,3,2],[1,5,6,89]])

In [239]:
A

array([[ 1,  2,  3,  4],
       [ 1,  3,  3,  2],
       [ 1,  5,  6, 89]])

In [250]:
X_train

array([[ 5.5,  2.4,  3.7,  1. ],
       [ 6.3,  2.8,  5.1,  1.5],
       [ 6.4,  3.1,  5.5,  1.8],
       [ 6.6,  3. ,  4.4,  1.4],
       [ 7.2,  3.6,  6.1,  2.5],
       [ 5.7,  2.9,  4.2,  1.3],
       [ 7.6,  3. ,  6.6,  2.1],
       [ 5.6,  3. ,  4.5,  1.5],
       [ 5.1,  3.5,  1.4,  0.2],
       [ 7.7,  2.8,  6.7,  2. ],
       [ 5.8,  2.7,  4.1,  1. ],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 5. ,  3.5,  1.3,  0.3],
       [ 5.1,  3.8,  1.9,  0.4],
       [ 5. ,  2. ,  3.5,  1. ],
       [ 6.3,  2.7,  4.9,  1.8],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 5.6,  2.7,  4.2,  1.3],
       [ 5.1,  3.4,  1.5,  0.2],
       [ 5.7,  3. ,  4.2,  1.2],
       [ 7.7,  3.8,  6.7,  2.2],
       [ 4.6,  3.2,  1.4,  0.2],
       [ 6.2,  2.9,  4.3,  1.3],
       [ 5.7,  2.5,  5. ,  2. ],
       [ 5.5,  4.2,  1.4,  0.2],
       [ 6. ,  3. ,  4.8,  1.8],
       [ 5.8,  2.7,  5.1,  1.9],
       [ 6. ,  2.2,  4. ,  1. ],
       [ 5

In [271]:
A=np.tile(X_train,(50000,1))

In [328]:
np.argwhere(np.logical_not((A==A[0]).all(axis=0)))

array([[0],
       [1],
       [2],
       [3]])

In [337]:
np.argwhere(np.logical_not((A==A[0]).all(axis=0))).flatten()

array([0, 1, 2, 3])

In [309]:
C=B


In [310]:
C

array([False, False, False, False], dtype=bool)

In [312]:
X_train[np.logical_not(C)]

array([ True,  True,  True,  True], dtype=bool)

In [286]:
X_train

array([[ 5.5,  2.4,  3.7,  1. ],
       [ 6.3,  2.8,  5.1,  1.5],
       [ 6.4,  3.1,  5.5,  1.8],
       [ 6.6,  3. ,  4.4,  1.4],
       [ 7.2,  3.6,  6.1,  2.5],
       [ 5.7,  2.9,  4.2,  1.3],
       [ 7.6,  3. ,  6.6,  2.1],
       [ 5.6,  3. ,  4.5,  1.5],
       [ 5.1,  3.5,  1.4,  0.2],
       [ 7.7,  2.8,  6.7,  2. ],
       [ 5.8,  2.7,  4.1,  1. ],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 5. ,  3.5,  1.3,  0.3],
       [ 5.1,  3.8,  1.9,  0.4],
       [ 5. ,  2. ,  3.5,  1. ],
       [ 6.3,  2.7,  4.9,  1.8],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 5.6,  2.7,  4.2,  1.3],
       [ 5.1,  3.4,  1.5,  0.2],
       [ 5.7,  3. ,  4.2,  1.2],
       [ 7.7,  3.8,  6.7,  2.2],
       [ 4.6,  3.2,  1.4,  0.2],
       [ 6.2,  2.9,  4.3,  1.3],
       [ 5.7,  2.5,  5. ,  2. ],
       [ 5.5,  4.2,  1.4,  0.2],
       [ 6. ,  3. ,  4.8,  1.8],
       [ 5.8,  2.7,  5.1,  1.9],
       [ 6. ,  2.2,  4. ,  1. ],
       [ 5

In [300]:
C = B.all(axis=0)

In [303]:
C

array([False, False, False, False], dtype=bool)

In [304]:

np.argwhere([True,True,False,True])

array([[0],
       [1],
       [3]])

In [299]:
X_train[:,[True,True,False,True]]

/home/xu/venv/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  if __name__ == '__main__':


array([[ 2.4,  2.4,  5.5,  2.4],
       [ 2.8,  2.8,  6.3,  2.8],
       [ 3.1,  3.1,  6.4,  3.1],
       [ 3. ,  3. ,  6.6,  3. ],
       [ 3.6,  3.6,  7.2,  3.6],
       [ 2.9,  2.9,  5.7,  2.9],
       [ 3. ,  3. ,  7.6,  3. ],
       [ 3. ,  3. ,  5.6,  3. ],
       [ 3.5,  3.5,  5.1,  3.5],
       [ 2.8,  2.8,  7.7,  2.8],
       [ 2.7,  2.7,  5.8,  2.7],
       [ 3.4,  3.4,  5.2,  3.4],
       [ 3.5,  3.5,  5. ,  3.5],
       [ 3.8,  3.8,  5.1,  3.8],
       [ 2. ,  2. ,  5. ,  2. ],
       [ 2.7,  2.7,  6.3,  2.7],
       [ 3.4,  3.4,  4.8,  3.4],
       [ 3. ,  3. ,  5. ,  3. ],
       [ 3.3,  3.3,  5.1,  3.3],
       [ 2.7,  2.7,  5.6,  2.7],
       [ 3.4,  3.4,  5.1,  3.4],
       [ 3. ,  3. ,  5.7,  3. ],
       [ 3.8,  3.8,  7.7,  3.8],
       [ 3.2,  3.2,  4.6,  3.2],
       [ 2.9,  2.9,  6.2,  2.9],
       [ 2.5,  2.5,  5.7,  2.5],
       [ 4.2,  4.2,  5.5,  4.2],
       [ 3. ,  3. ,  6. ,  3. ],
       [ 2.7,  2.7,  5.8,  2.7],
       [ 2.2,  2.2,  6. ,  2.2],
       [ 3

In [281]:
X_train.shape

(105, 4)

In [284]:
np.diff(X_train,axis=0)

array([[ 0.8,  0.4,  1.4,  0.5],
       [ 0.1,  0.3,  0.4,  0.3],
       [ 0.2, -0.1, -1.1, -0.4],
       [ 0.6,  0.6,  1.7,  1.1],
       [-1.5, -0.7, -1.9, -1.2],
       [ 1.9,  0.1,  2.4,  0.8],
       [-2. ,  0. , -2.1, -0.6],
       [-0.5,  0.5, -3.1, -1.3],
       [ 2.6, -0.7,  5.3,  1.8],
       [-1.9, -0.1, -2.6, -1. ],
       [-0.6,  0.7, -2.7, -0.8],
       [-0.2,  0.1, -0.1,  0.1],
       [ 0.1,  0.3,  0.6,  0.1],
       [-0.1, -1.8,  1.6,  0.6],
       [ 1.3,  0.7,  1.4,  0.8],
       [-1.5,  0.7, -3. , -1.6],
       [ 0.2, -0.4, -0.3,  0. ],
       [ 0.1,  0.3,  0.1,  0.3],
       [ 0.5, -0.6,  2.5,  0.8],
       [-0.5,  0.7, -2.7, -1.1],
       [ 0.6, -0.4,  2.7,  1. ],
       [ 2. ,  0.8,  2.5,  1. ],
       [-3.1, -0.6, -5.3, -2. ],
       [ 1.6, -0.3,  2.9,  1.1],
       [-0.5, -0.4,  0.7,  0.7],
       [-0.2,  1.7, -3.6, -1.8],
       [ 0.5, -1.2,  3.4,  1.6],
       [-0.2, -0.3,  0.3,  0.1],
       [ 0.2, -0.5, -1.1, -0.9],
       [-0.6,  0.8,  0.5,  0.5],
       [ 0

In [272]:
A[:,((A==A[0]).T).sum(axis=1)!=A.shape[1]]

array([[ 5.5,  2.4,  3.7,  1. ],
       [ 6.3,  2.8,  5.1,  1.5],
       [ 6.4,  3.1,  5.5,  1.8],
       ..., 
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.8,  2.6,  4. ,  1.2],
       [ 7.1,  3. ,  5.9,  2.1]])

In [233]:
X_train

array([[ 5.5,  2.4,  3.7,  1. ],
       [ 6.3,  2.8,  5.1,  1.5],
       [ 6.4,  3.1,  5.5,  1.8],
       [ 6.6,  3. ,  4.4,  1.4],
       [ 7.2,  3.6,  6.1,  2.5],
       [ 5.7,  2.9,  4.2,  1.3],
       [ 7.6,  3. ,  6.6,  2.1],
       [ 5.6,  3. ,  4.5,  1.5],
       [ 5.1,  3.5,  1.4,  0.2],
       [ 7.7,  2.8,  6.7,  2. ],
       [ 5.8,  2.7,  4.1,  1. ],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 5. ,  3.5,  1.3,  0.3],
       [ 5.1,  3.8,  1.9,  0.4],
       [ 5. ,  2. ,  3.5,  1. ],
       [ 6.3,  2.7,  4.9,  1.8],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 5.6,  2.7,  4.2,  1.3],
       [ 5.1,  3.4,  1.5,  0.2],
       [ 5.7,  3. ,  4.2,  1.2],
       [ 7.7,  3.8,  6.7,  2.2],
       [ 4.6,  3.2,  1.4,  0.2],
       [ 6.2,  2.9,  4.3,  1.3],
       [ 5.7,  2.5,  5. ,  2. ],
       [ 5.5,  4.2,  1.4,  0.2],
       [ 6. ,  3. ,  4.8,  1.8],
       [ 5.8,  2.7,  5.1,  1.9],
       [ 6. ,  2.2,  4. ,  1. ],
       [ 5

In [231]:
X_train.T==X_train.T[0]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, Fals

In [221]:
vecfunc

In [219]:
X_train==X_train[]

array([[ True,  True,  True,  True],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False,  True],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False,  True],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [ True, False, False, False],
 

In [214]:
np.count_nonzero(X_train[])

420

In [211]:
np.unique(X_train[:,3])

array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  1. ,  1.1,  1.2,  1.3,  1.4,
        1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,  2.3,  2.4,  2.5])

In [169]:
X_train[X_train[:,1]>]

array([[ 7.2,  3.6,  6.1,  2.5],
       [ 5.1,  3.8,  1.9,  0.4],
       [ 7.7,  3.8,  6.7,  2.2],
       [ 5.5,  4.2,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 5.1,  3.8,  1.6,  0.2],
       [ 5.3,  3.7,  1.5,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 5.8,  4. ,  1.2,  0.2]])

In [172]:
b = (X_train[:,1]>3)

In [173]:
b.

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True], dtype=bool)

In [176]:
np.zeros([1,2,3])

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [198]:
X_train[np.nonzero(X_train[:,1]>3)[0]].shape

(41, 4)

In [178]:
X_train[:,1]>3

array([False, False,  True, False,  True, False, False, False,  True,
       False, False,  True,  True,  True, False, False,  True, False,
        True, False,  True, False,  True,  True, False, False,  True,
       False, False, False, False,  True, False,  True, False, False,
        True,  True, False, False,  True, False,  True,  True,  True,
       False,  True, False, False, False, False, False, False, False,
        True,  True,  True, False, False,  True,  True, False,  True,
        True, False, False, False,  True,  True,  True,  True, False,
       False, False, False, False, False, False, False,  True, False,
       False,  True,  True, False,  True, False,  True,  True, False,
        True, False, False, False, False, False, False, False, False,
        True, False, False,  True, False, False], dtype=bool)

In [51]:
np.where()X_train

array([[ 5.5,  2.4,  3.7,  1. ],
       [ 6.3,  2.8,  5.1,  1.5],
       [ 6.4,  3.1,  5.5,  1.8],
       [ 6.6,  3. ,  4.4,  1.4],
       [ 7.2,  3.6,  6.1,  2.5],
       [ 5.7,  2.9,  4.2,  1.3],
       [ 7.6,  3. ,  6.6,  2.1],
       [ 5.6,  3. ,  4.5,  1.5],
       [ 5.1,  3.5,  1.4,  0.2],
       [ 7.7,  2.8,  6.7,  2. ],
       [ 5.8,  2.7,  4.1,  1. ],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 5. ,  3.5,  1.3,  0.3],
       [ 5.1,  3.8,  1.9,  0.4],
       [ 5. ,  2. ,  3.5,  1. ],
       [ 6.3,  2.7,  4.9,  1.8],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 5.6,  2.7,  4.2,  1.3],
       [ 5.1,  3.4,  1.5,  0.2],
       [ 5.7,  3. ,  4.2,  1.2],
       [ 7.7,  3.8,  6.7,  2.2],
       [ 4.6,  3.2,  1.4,  0.2],
       [ 6.2,  2.9,  4.3,  1.3],
       [ 5.7,  2.5,  5. ,  2. ],
       [ 5.5,  4.2,  1.4,  0.2],
       [ 6. ,  3. ,  4.8,  1.8],
       [ 5.8,  2.7,  5.1,  1.9],
       [ 6. ,  2.2,  4. ,  1. ],
       [ 5

In [58]:
(X_train[0] == X_train[0]).all()

True

In [62]:
X_train[0].repeat(,axis=)

ValueError: axis(=1) out of bounds

In [65]:
(np.tile(X_train[0],(4,1))== X_train[0]).all()

True

In [68]:
np.tile(X_train[0],(4,1))

array([[ 5.5,  2.4,  3.7,  1. ],
       [ 5.5,  2.4,  3.7,  1. ],
       [ 5.5,  2.4,  3.7,  1. ],
       [ 5.5,  2.4,  3.7,  1. ]])

In [114]:
np.tile(X_train,(500,1)).shape

(52500, 4)

In [112]:
(np.tile(X_train,(500,1))==X_train[0]).all()

False

In [120]:
(np.diff(np.tile(X_train[0],(4,1)),1,0)==0).all()

True

In [133]:
np.diff(np.tile(X_train[0],(4,1)),,0)==0

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]], dtype=bool)

In [81]:
X_train.shape

(105, 4)